# CAFA 5 protein function Prediction with TensorFlow

This notebook walks you through how to train a DNN model using TensorFlow on the CAFA 5 protein function Prediction dataset made available for this competition.

The objective of the model is to predict the function(aka **GO term ID**) of a set of proteins based on their amino acid sequences and other data.


**Note** : This notebook runs without any GPU. This is because enabling GPUs leaves less RAM memory on the VM and the submission step needs a lot of memory. One point where this would impact is when training the model. With CPU it will take around 2 minutes while on GPU it would take around 30 seconds.

## About the Data

### Protein Sequence

Each protein is composed of dozens or hundreds of amino acids that are linked sequentially. Each amino acid in the sequence may be represented by a one-letter or three-letter code. Thus the sequence of a protein is often notated as a string of letters.

<img src="https://cityu-bioinformatics.netlify.app/img/tools/protein/pro_seq.png" alt ="Sequence.png" style='width: 800px;' >

Image source - [https://cityu-bioinformatics.netlify.app/](https://cityu-bioinformatics.netlify.app/too2/new_proteo/pro_seq/)

The `train_sequences.fasta` made available for this competitions, contains the sequences for proteins with annotations (labelled proteins).

# Gene Ontology

We can define the functional properties of a proteins using Gene Ontology(GO). Gene Ontology (GO) describes our understanding of the biological domain with respect to three aspects:
1. Molecular Function (MF)
2. Biological Process (BP)
3. Cellular Component (CC)

Read more about Gene Ontology [here](http://geneontology.org/docs/ontology-documentation).

File `train_terms.tsv` contains the list of annotated terms (ground truth) for the proteins in `train_sequences.fasta`. In `train_terms.tsv` the first column indicates the protein's UniProt accession ID (unique protein id), the second is the `GO Term ID`, and the third indicates in which ontology the term appears.

# Labels of the dataset

The objective of our model is to predict the terms (functions) of a protein sequence. One protein sequence can have many functions and can thus be classified into any number of terms. Each term is uniquely identified by a `GO Term ID`. Thus our model has to predict all the `GO Term ID`s for a protein sequence. This means that the task at hand is a multi-label classification problem.

# Protein embeddings for train and test data

To train a machine learning model we cannot use the alphabetical protein sequences in`train_sequences.fasta` directly. They have to be converted into a vector format. In this notebook, we will use embeddings of the protein sequences to train the model. You can think of protein embeddings to be similar to word embeddings used to train NLP models.
<!-- Instead, to make calculations and data preparation easier we will use precalculated protein embeddings.
 -->
Protein embeddings are a machine-friendly method of capturing the protein's structural and functional characteristics, mainly through its sequence. One approach is to train a custom ML model to learn the protein embeddings of the protein sequences in the dataset being used in this notebook. Since this dataset represents proteins using amino-acid sequences which is a standard approach, we can use any publicly available pre-trained protein embedding models to generate the embeddings.

There are a variety of protein embedding models. To make data preparation easier, we have used the precalculated protein embeddings created by [Sergei Fironov](https://www.kaggle.com/sergeifironov) using the Rost Lab's T5 protein language model in this notebook. The precalculated protein embeddings can be found [here](https://www.kaggle.com/datasets/sergeifironov/t5embeds). We have added this dataset to the notebook along with the dataset made available for the competition.

To add this to your enviroment, on the right side panel, click on `Add Data` and search for `t5embeds` (make sure that it's the correct [one](https://www.kaggle.com/datasets/sergeifironov/t5embeds)) and then click on the `+` beside it.

**We'll start by introducing a variable that decides whether we prepare the training data cell by cell or if we do it all at once (for memory issues)**


In [1]:
notebook_run = False

# Import the Required Libraries

In [2]:
import torch
torch.version.cuda

'11.8'

In [3]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['CUDA'] = torch.version.cuda
print(torch.__version__)
print(torch.version.cuda)

!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0%2Bcu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0%2Bcu118.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torchmetrics
!pip install networkx Bio

2.0.0
11.8
Looking in links: https://data.pyg.org/whl/torch-2.0.0%2Bcu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 33.9 MB/s eta 0:00:0000:010:01
Looking in links: https://data.pyg.org/whl/torch-2.0.0%2Bcu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 61.5 MB/s eta 0:00:0000:0100:01
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-022ad93p
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-022ad93p
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 87ca71913efa164efe07622b5e1fcd189125d39c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=971733 sha256=b9691a5991acbcf32e631fd60de595f2cc576c4b9afb5c1ef609917f5acea6fa
  Stored in directory: /tmp/pip-ep

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from scipy.sparse import coo_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool, GCNConv, global_max_pool, global_mean_pool, GATConv, SAGEConv
from torch_geometric.data import Data, Batch
from torch_geometric.loader.dataloader import DataLoader
from torch.utils.data import Dataset
from torchmetrics import Accuracy

from torch_geometric.utils import from_networkx, to_networkx, to_dense_batch, convert
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU, SELU
from torch_geometric.nn.models import GAT
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch_geometric.nn.models as models
from Bio import SeqIO
from sklearn.metrics import classification_report
import concurrent.futures

# Required for progressbar widget
import progressbar
from datetime import datetime
import random
random.seed(datetime.now().timestamp())

/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:39: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:70: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libt

In [5]:
print("Numpy v" + np.__version__)
kaggle_input_data = '/kaggle/input/cafa-5-protein-function-prediction'

Numpy v1.23.5


# Load the Dataset

First we will load the file `train_terms.tsv` which contains the list of annotated terms (functions) for the proteins. We will extract the labels aka `GO term ID` and create a label dataframe for the protein embeddings.

In [6]:
if notebook_run:
    #train_terms = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv",sep="\t")
    train_terms = pd.read_csv(kaggle_input_data + "/Train/train_terms.tsv",sep="\t")
    print(train_terms.shape)

`train_terms` dataframe is composed of 3 columns and 5363863 entries. We can see all 3 dimensions of our dataset by printing out the first 5 entries using the following code:

In [7]:
if notebook_run:
    train_terms = train_terms.sample(150000)

In [8]:
if notebook_run:
    train_terms.loc[train_terms['EntryID'] == 'Q59VP0']

If we look at the first entry of `train_terms.tsv`, we can see that it contains protein id(`A0A009IHW8`), the GO term(`GO:0008152`) and its aspect(`BPO`).

# A different kind of embedding

We introduce a new embedding for Proteins based on the notion of Recurrence Networks: Olyaee, M. H., Yaghoubi, A., & Yaghoobi, M. (2016). Predicting protein structural classes based on complex networks and recurrence analysis. Journal of theoretical biology, 404, 375–382. https://doi.org/10.1016/j.jtbi.2016.06.018

Instead of using the provided embeddings, we will train Graph Neural Networks on a new embedding for each graph, defined by recurrence networks.

**Let's start by defining a function that takes a Protein Sequence String and creates a recurrence network:**

In [9]:
from Bio.SeqUtils.ProtParamData import *
from collections import Counter

def protein_recurrence_network(sequence):
    # Create an empty graph for the recurrence network
    graph = nx.DiGraph()

    # Use a Counter to store the edge weights
    edge_weights = Counter(zip(sequence, sequence[1:]))

    # Add nodes and edges to the graph based on the edge weights using list comprehension
    graph.add_weighted_edges_from(((char1, char2, weight / len(sequence)) 
                                    for (char1, char2), weight in edge_weights.items()))
    return protein_attributes(graph)


def protein_attributes(graph):
    nx.set_node_attributes(graph, nx.betweenness_centrality(graph), "betweenness")

     # Create a dictionary of attribute values for all nodes
    attributes = {}
    for node in graph.nodes:
        att_dict = {
            k: v[node] if node in v else 0. for k, v in gravy_scales.items()
        }
        att_dict.update({
            'flexibility': Flex.get(node, 0.),
            'hydrophilicity': hw.get(node, 0.),
            'surface_accessibility': em.get(node, 0.),
            'janin_interior': ja.get(node, 0.)
        })
        attributes[node] = att_dict

    # Update attributes for all nodes in a single step
    nx.set_node_attributes(graph, attributes)

    # Now setup interaction details
    for v1,v2 in graph.edges:
        try:
            graph[v1][v2]['instability_index'] = DIWV[v1][v2]
        except KeyError:
            graph[v1][v2]['instability_index'] = 0.

    return graph

In [10]:
# Define a function to process a record and return the protein recurrence network
#def process_record(record):
#    return record.id, protein_recurrence_network(str(record.seq))
def process_record(record):
    if record.id in train_terms_set:
        return record.id, protein_recurrence_network(str(record.seq))
    else:
        return None

if notebook_run:
    train_terms_set = set(train_terms['EntryID'].values)
    
    # Parse the fasta file
    records = SeqIO.parse(kaggle_input_data + "/Train/train_sequences.fasta", "fasta")

    # Use map to process the records in parallel
    protein_rn_dict = dict(map(process_record, (record for record in records if record.id in train_terms['EntryID'].values)))
    g_train_protein_ids = np.array(list(protein_rn_dict.keys()))
    
    # Use concurrent.futures to process the records in parallel
    #with concurrent.futures.ProcessPoolExecutor() as executor:
    #    future_to_record = {executor.submit(process_record, record): record for record in records}
    #    protein_rn_dict = {future.result()[0]: future.result()[1] for future in concurrent.futures.as_completed(future_to_record) if future.result() is not None}
    #    g_train_protein_ids = np.array(list(protein_rn_dict.keys()))

Let's draw one of these graphs to see what it looks like.

In [11]:
if notebook_run:
    G_id = train_terms['EntryID'].sample(1).values[0]
    G = protein_rn_dict[G_id]
    fig = plt.figure(figsize=(12,12))
    # Extract edge weights
    edge_weights = nx.get_edge_attributes(G, 'weight')
    # Round the edge weights to 4 decimal places
    edge_weights = {k: round(v, 4) for k, v in edge_weights.items()}

    # Draw the graph
    pos = nx.spring_layout(G)
    nx.draw_networkx(G, pos, with_labels=True, node_color='lightblue')

    # Draw edge labels
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_weights, font_color='red', font_size=8)
    plt.title(G_id)

    # Show the graph
    plt.axis('off')
    plt.show()

In [12]:
if notebook_run:
    list(G.nodes(data=True))
    #for v1,v2 in G.edges:
    #  print(v1)
    #  print(v2)

In [13]:
if notebook_run:
    # Just looking for weird proteins
    for k in protein_rn_dict:
        if 'U' in protein_rn_dict[k].nodes():
            print(k)
            print(protein_rn_dict[k].nodes())
            print(protein_rn_dict[k])

# Prepare the dataset

Reference: https://www.kaggle.com/code/alexandervc/baseline-multilabel-to-multitarget-binary

First we will extract all the needed labels(`GO term ID`) from `train_terms.tsv` file. There are more than 40,000 labels. In order to simplify our model, we will choose the most frequent 1500 `GO term ID`s as labels.

Let's plot the most frequent 100 `GO Term ID`s in `train_terms.tsv`.

In [14]:
if notebook_run:
    # Select first 1500 values for plotting
    plot_df = train_terms['term'].value_counts().iloc[:100]

    figure, axis = plt.subplots(1, 1, figsize=(12, 6))

    bp = sns.barplot(ax=axis, x=np.array(plot_df.index), y=plot_df.values)
    bp.set_xticklabels(bp.get_xticklabels(), rotation=90, size = 6)
    axis.set_title('Top 100 frequent GO term IDs')
    bp.set_xlabel("GO term IDs", fontsize = 12)
    bp.set_ylabel("Count", fontsize = 12)
    plt.show()

We will now save the first 1500 most frequent GO term Ids into a list.

In [15]:
if notebook_run:
    # Set the limit for label
    num_of_labels = 1500

    # Take value counts in descending order and fetch first 1500 `GO term ID` as labels
    labels = train_terms['term'].value_counts().index[:num_of_labels].tolist()

Next, we will create a new dataframe by filtering the train terms with the selected `GO Term ID`s.

In [16]:
if notebook_run:
    # Fetch the train_terms data for the relevant labels only
    train_terms_updated = train_terms.loc[train_terms['term'].isin(labels)]

Let us plot the aspect values in the new **train_terms_updated** dataframe using a pie chart.

In [17]:
if notebook_run:
    pie_df = train_terms_updated['aspect'].value_counts()
    palette_color = sns.color_palette('bright')
    plt.pie(pie_df.values, labels=np.array(pie_df.index), colors=palette_color, autopct='%.0f%%')
    plt.show()

As you can see, majority of the `GO term Id`s have BPO(Biological Process Ontology) as their aspect.

Since this is a multi label classification problem, in the labels array we will denote the presence or absence of each Go Term Id for a protein id using a 1 or 0.
First, we will create a numpy array `train_labels` of required size for the labels. To update the `train_labels` array with the appropriate values, we will loop through the label list.

In [18]:
if notebook_run:
    # Setup progressbar settings.
    # This is strictly for aesthetic.
    bar = progressbar.ProgressBar(maxval=num_of_labels, \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    # Create an empty dataframe of required size for storing the labels,
    # i.e, train_size x num_of_labels (142246 x 1500)
    train_size = g_train_protein_ids.shape[0] # len(X)
    train_labels = np.zeros((train_size, num_of_labels))

    # Convert from numpy to pandas series for better handling
    series_train_protein_ids = pd.Series(g_train_protein_ids)

    # Generate a dict where key is label and value is the corresponding proteins
    def get_label_proteins(label):
        proteins = train_terms_updated[train_terms_updated['term'] == label]['EntryID'].unique()
        return label, proteins

    label_protein_dict = dict(map(get_label_proteins, labels))

    # Loop through each label
    for i, label in enumerate(labels):
        # Fetch all the unique EntryId aka proteins related to the current label(GO term ID)
        label_related_proteins = label_protein_dict[label]

        # In the series_train_protein_ids pandas series, if a protein is related
        # to the current label, then mark it as 1, else 0.
        # Replace the ith column of train_labels with that pandas series.
        train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)

        # Progress bar percentage increase
        bar.update(i+1)

    # Notify the end of progress bar
    bar.finish()

    # Convert train_labels numpy into pandas dataframe
    labels_df = pd.DataFrame(data = train_labels, columns = labels)
    print(labels_df.shape)


The final labels dataframe (`label_df`) is composed of 1500 columns and 142246 entries (though we brought it down to 10000). We can see all 1500 dimensions(results will be truncated since the number of columns is big) of our dataset by printing out the first 5 entries using the following code:

In [19]:
if notebook_run:
    labels_df.head()

In [20]:
if notebook_run:
    num_of_labels

## Graph Preparation

Next, we'll define our GNN model class, which will inherit from nn.Module:

In [21]:
import torch_geometric
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels=1, hidden_dim = 128, out_channels=1500):
        """
        Represents a 3-layer GraphSAGE GNN model
        with embedding and hidden dimension of hidden_dim.
        """
        super(GraphSAGE, self).__init__()
        self.tag = 'GraphSAGE'
        self.pre_embs = nn.Embedding(in_channels, hidden_dim)

        self.convs = nn.ModuleList()
        self.selus = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        self.layer_norms = nn.ModuleList()

        # Input layer
        self.convs.append(SAGEConv(in_channels, hidden_dim))
        self.selus.append(nn.SELU())
        self.dropouts.append(nn.Dropout(0.1))
        self.layer_norms.append(nn.LayerNorm(hidden_dim))

        # Hidden layers
        for _ in range(3):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
            self.selus.append(SELU())
            self.dropouts.append(nn.Dropout(0.1))
            self.layer_norms.append(nn.LayerNorm(hidden_dim))

        # Output layer
        self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        self.out = nn.Linear(3*hidden_dim, out_channels)
        self.out.weight.data.fill_(1.)
        self.dropout = nn.Dropout(0.1)
            
    def forward(self, data):
        """
        Runs a forward pass through GraphSAGE with given initial skill IDs and
        edge_index and edge_weights.

        Arguments:
          - x: amino acid IDs (torch.Tensor)
          - edge_index: edges in skill graph (torch.Tensor)
          - edge_weight: edge weights of skill graph (torch.Tensor)

        Returns:
          - final node embedding for skill
        """
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # Convert x tensor to Long type
        #x = x.to(torch.long)
        #x = self.pre_embs(x)
        for conv, selu, dropout, layer_norm in zip(self.convs, self.selus, self.dropouts, self.layer_norms):
          x = layer_norm(dropout(selu(conv(x, edge_index))))

        x_add = global_add_pool(x, batch)
        x_mean = global_mean_pool(x, batch)
        x_max = global_max_pool(x, batch)
        x = torch.cat([x_add, x_mean, x_max], dim=-1)
        return self.out(x)

class GraphGAT(nn.Module):
    def __init__(self, in_channels, hidden_dim, num_layers, out_channels, dropout, act):
        super(GraphGAT, self).__init__()
        self.tag = 'GraphGAT'
        self.pre_embs = nn.Embedding(in_channels, hidden_dim)

        self.convs = nn.ModuleList()
        self.selus = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        self.layer_norms = nn.ModuleList()

        # Input layer
        self.convs.append(GATConv(in_channels, hidden_dim))
        self.selus.append(nn.SELU())
        self.dropouts.append(nn.Dropout(dropout))
        self.layer_norms.append(nn.LayerNorm(hidden_dim))

        # Hidden layers
        for _ in range(num_layers):
            self.convs.append(GATConv(hidden_dim, hidden_dim))
            self.selus.append(SELU())
            self.dropouts.append(nn.Dropout(dropout))
            self.layer_norms.append(nn.LayerNorm(hidden_dim))

        # Output layer
        self.convs.append(GATConv(hidden_dim, hidden_dim))
        self.out = nn.Linear(3*hidden_dim, out_channels)
        self.out.weight.data.fill_(1.)
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        """Runs a forward pass through GraphGAT with given initial IDs and
        edge_index and edge_weights.

        Arguments:
          - x: amino acid IDs (torch.Tensor)
          - edge_index: edges in skill graph (torch.Tensor)
          - edge_weight: edge weights of skill graph (torch.Tensor)

        Returns:
          - final node embedding for skill
        """
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # Convert x tensor to Long type
        #x = x.to(torch.long)
        #x = self.pre_embs(x)
        for conv, selu, dropout, layer_norm in zip(self.convs, self.selus, self.dropouts, self.layer_norms):
          x = layer_norm(dropout(selu(conv(x, edge_index))))

        x_add = global_add_pool(x, batch)
        x_mean = global_mean_pool(x, batch)
        x_max = global_max_pool(x, batch)
        x = torch.cat([x_add, x_mean, x_max], dim=-1)
        return self.out(x)


Here, **hidden_dim** is the dimensionality of hidden layers, and **output_dim** is the number of output classes (1500 in this case).

Now, let's define a function to convert each NetworkX graph into a PyTorch Geometric **Data** object:

In [22]:
# Assuming nx_graphs_train and nx_graphs_test are your lists of NetworkX graphs
# and labels_train and labels_test are your lists of labels for train and test datasets respectively
#TODO Use map for this and do it with singles
def from_networkx_to_data(nx_graph, labels):
    data = from_networkx(nx_graph)

    # Extract node properties and convert them to a feature tensor
    data.x = torch.tensor([[float(nx_graph.nodes[node][prop]) for prop in nx_graph.nodes[node]]
                            for node in nx_graph.nodes()], dtype=torch.float)
    
    data.y = torch.tensor(labels, dtype=torch.float).unsqueeze(0)
    return data
    
def from_networkx_to_data_list(nx_graphs, labels):
    return list(map(from_networkx_to_data, nx_graphs,labels))

In this function, we convert the adjacency matrix of the graph into the edge index format expected by PyTorch Geometric. We also convert the labels into a torch tensor.

# Training

Now, let's define the training loop and train the GNN model on your labeled graphs:

In [23]:
import random
from datetime import datetime
random.seed(datetime.now().timestamp())
if notebook_run:
    # Assuming you have a list of NetworkX graphs and corresponding labels

    #protein_ids = list(protein_rn_dict.keys())
    protein_ids = g_train_protein_ids
    num_features = 33
    num_classes = 1500
    graphs, graphs_test, labels, labels_test = train_test_split([protein_rn_dict[k] for k in protein_ids],
                                                                labels_df.iloc[np.where(np.isin(g_train_protein_ids, protein_ids))].values,
                                                                test_size=0.2, random_state=int(datetime.now().timestamp()))

    # Convert each graph to a PyTorch Geometric Data object and place in data loader
    train_loader = DataLoader(from_networkx_to_data_list(graphs, labels), batch_size=256, shuffle=True)
    test_loader = DataLoader(from_networkx_to_data_list(graphs_test, labels_test), batch_size=256, shuffle=False)

We'll introduce a method to do all of this data preparation at once, such that we don't store all of these intermediate variables in memory.

In [39]:
class ProteinDataset(Dataset):
    def __init__(self, protein_sequences, labels=torch.zeros((1500,1)), training=False):
        self.protein_sequences = protein_sequences
        self.labels = labels
        self.training = training
        if training:
            self.dataset = list(map(from_networkx_to_data, map(protein_recurrence_network, protein_sequences), self.labels))

    def __len__(self):
        return len(self.protein_sequences)

    def __getitem__(self, idx):
        
        # Convert the NetworkX graph to a PyTorch Geometric Data object
        if self.training:
            data = self.dataset[idx]
        else:
            data = from_networkx_to_data(protein_recurrence_network(self.protein_sequences[idx]), self.labels[idx])
        return data

def prepare_training_data_and_models(sample_data_size=150000, num_of_labels=1500, training=True, just_data=False):
    '''
        Inputs:
            sample_data_size - How many samples should be used in the training set
            num_of_labels - Limit of how many lables (most frequently occurring) should we include
    '''
    train_terms = pd.read_csv(kaggle_input_data + "/Train/train_terms.tsv",sep="\t")
    print("Training terms shape: ")
    print(train_terms.shape)
    if sample_data_size:
        train_terms = train_terms.sample(sample_data_size)

    print("Getting record IDs")
    # Parse the fasta file
    records = SeqIO.parse(kaggle_input_data + "/Train/train_sequences.fasta", "fasta")
    entry_id_set = set(train_terms['EntryID'].values)
    train_protein_ids = np.array([record.id for record in records if record.id in entry_id_set])
    print(str(len(train_protein_ids)) + " Protein IDs found")

    print("Fetching Label Values")
    # Take value counts in descending order and fetch first 1500 `GO term ID` as labels
    labels = train_terms['term'].value_counts().index[:num_of_labels].tolist()
    
    # Fetch the train_terms data for the relevant labels only
    train_terms_updated = train_terms.loc[train_terms['term'].isin(labels)]
    
    print("Establishing labels")
    
    # Setup progressbar settings.
    # This is strictly for aesthetic.
    bar = progressbar.ProgressBar(maxval=num_of_labels, \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    # Create an empty dataframe of required size for storing the labels,
    # i.e, train_size x num_of_labels (142246 x 1500)
    train_size = train_protein_ids.shape[0] # len(X)
    train_labels = np.zeros((train_size, num_of_labels))

    # Convert from numpy to pandas series for better handling
    series_train_protein_ids = pd.Series(train_protein_ids)

    # Generate a dict where key is label and value is the corresponding proteins
    def get_label_proteins(label):
        proteins = train_terms_updated[train_terms_updated['term'] == label]['EntryID'].unique()
        return label, proteins
    
    print("Creating Label-Protein Dictionary")

    label_protein_dict = dict(map(get_label_proteins, labels))

    # Loop through each label
    for i, label in enumerate(labels):
        # Fetch all the unique EntryId aka proteins related to the current label(GO term ID)
        label_related_proteins = label_protein_dict[label]

        # In the series_train_protein_ids pandas series, if a protein is related
        # to the current label, then mark it as 1, else 0.
        # Replace the ith column of train_labels with that pandas series.
        train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)

        # Progress bar percentage increase
        bar.update(i+1)

    # Notify the end of progress bar
    bar.finish()

    # Convert train_labels numpy into pandas dataframe
    labels_df = pd.DataFrame(data = train_labels, columns = labels)

    #protein_ids = list(protein_rn_dict.keys())
    protein_ids = train_protein_ids
    num_features = 33
    num_classes = num_of_labels
    proteins_train, proteins_test, labels, labels_test = train_test_split(protein_ids,
                                                                labels_df.iloc[np.where(np.isin(train_protein_ids, protein_ids))].values,
                                                                test_size=0.2, random_state=int(datetime.now().timestamp()))
    print("Getting sequences for training")
    records = SeqIO.parse(kaggle_input_data + "/Train/train_sequences.fasta", "fasta")
    proteins_train_set = set(proteins_train)
    training_dataset = ProteinDataset([str(record.seq) for record in records if record.id in proteins_train_set], labels, training=training)
    print("Getting sequences for testing")
    records = SeqIO.parse(kaggle_input_data + "/Train/train_sequences.fasta", "fasta")
    proteins_test_set = set(proteins_test)
    test_dataset = ProteinDataset([str(record.seq) for record in records if record.id in proteins_test_set], labels_test, training=training)
    
    print("Putting training data into Data Loader")
    # Convert each graph to a PyTorch Geometric Data object and place in data loader
    train_loader = DataLoader(training_dataset, batch_size=512, shuffle=True, num_workers=2, pin_memory=True)
    print("Putting test data into Data Loader")
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=2, pin_memory=True)
    
    # Prepare Model
    print("Preparing model")
    def weights_init(m):
        if isinstance(m, nn.Linear):
            m.weight.data.fill_(1.)

    if just_data:
        return train_loader, test_loader
    else:
        # Calculate the class frequencies (number of occurrences) for each label
        class_frequencies = labels_df.sum(axis=0)

        # Calculate the total number of samples (rows)
        total_samples = len(labels_df)

        # Calculate the class weights as the inverse of class frequencies
        class_weights = total_samples / (class_frequencies * len(labels_df.columns))

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        #model = GINNet(num_features, 256, num_classes).to(device)
        model = GraphGAT(in_channels=num_features,hidden_dim=128,num_layers=5,out_channels=num_classes,dropout=0.1,act='selu').to(device)
        #model = GraphSAGE(num_features, 128, num_classes).to(device)
        # Initialize final layer to 0s
        model.apply(weights_init)
        hidden_channels = 64
        num_layers = 30
        #model = GCN(num_features, 256, num_classes).to(device)
        #model = GraphUnet(num_features, num_classes, n_hidden=128)
        #model = models.GraphSAGE(num_features,hidden_channels,num_layers,num_classes,dropout=0.1,act='selu',act_first=True).to(device)
        return train_loader, test_loader, model, device, class_weights

In [ ]:
if not notebook_run:
    train_loader, test_loader, model, device, class_weights = prepare_training_data_and_models(sample_data_size=10000)

Training terms shape: 
(5363863, 3)
Getting record IDs
9307 Protein IDs found
Fetching Label Values
Establishing labels
Creating Label-Protein Dictionary


[========================================================================] 100%


Getting sequences for training


In [ ]:
for i,data in enumerate(train_loader):
    print(f'Step {i + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    
    if i > 10:
        break

In [ ]:
import torch

# After running each iteration:
torch.cuda.empty_cache()

In [ ]:
from IPython.display import clear_output
from torchmetrics.classification import MultilabelStatScores, MultilabelAccuracy
import torch.nn.functional as F
import time


if notebook_run:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #model = GINNet(num_features, 256, num_classes).to(device)
    model = GraphSAGE(num_features, 256, num_classes).to(device)

    hidden_channels = 64
    num_layers = 30
    #model = GCN(num_features, 256, num_classes).to(device)
    #model = GraphUnet(num_features, num_classes, n_hidden=128)
    #model = models.GraphSAGE(num_features,hidden_channels,num_layers,num_classes,dropout=0.1,act='selu',act_first=True).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
loss_function = torch.nn.MultiLabelSoftMarginLoss(weight = torch.Tensor(class_weights.values).to(device) )
accuracy = MultilabelAccuracy(num_labels=1500).to(device)
mlss = MultilabelStatScores(1500, average='micro').to(device)

def train():
    model.train()

    correct = 0
    total_loss = 0
    for i,data in enumerate(train_loader):
        data = data.to(device)
        out = model(data)
        loss = loss_function(out, data.y)
        loss.backward()  # Backward pass (calculate gradients)

        optimizer.step()
        optimizer.zero_grad()
        
        total_loss += loss
            
    # After the training loop ends, there might be gradients that are not yet updated
    # So, you should perform an optimization step outside the loop to update those gradients
    optimizer.step()

    return total_loss / len(train_loader.dataset)  # Derive ratio of correct predictions.

def test(data_loader, verbose=True):
    model.eval()

    #all_preds = []
    #all_labels = []
    total_acc = 0
    clear_output(wait=True)

    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            out = model(data)
            total_acc += accuracy(out,data.y)
            if verbose:
                print('[tp, fp, tn, fn, sup]')
                print(mlss(out,data.y))
    #if verbose:
    #    print(classification_report(all_labels, all_preds))
    return total_acc / len(data_loader.dataset)  # Derive ratio of correct predictions.

    
num_folds = 500
num_epochs = 100
losses = []
test_accs = []
epochs = []

for k in range(num_folds):
    print("Fold: " + str(k+1))
    for epoch in range(1, num_epochs):
        start = time.time()
        loss = train().detach().cpu()
        test_acc = test(test_loader).detach().cpu()
        end = time.time()
    
        # Update the loss, test_acc, and epochs variables
        losses.append(loss)
        test_accs.append(test_acc)
        epochs.append(epoch)

        # Plot the updated data
        plt.plot(epochs, losses, label='Loss')
        plt.plot(epochs, test_accs, label='Test Accuracy')

        # Add labels and a legend to the plot
        plt.xlabel('Epoch')
        plt.ylabel('Value')
        plt.legend()

        # Label the last loss and test_acc values
        last_loss = round(losses[-1].numpy().tolist(), 4)
        last_acc = round(test_accs[-1].numpy().tolist(), 4)
        plt.text(epochs[-1], losses[-1], f'{last_loss}', ha='right', va='bottom')
        plt.text(epochs[-1], test_accs[-1], f'{last_acc}', ha='right', va='bottom')

        print(f'Epoch: {epoch:03d}, Train Loss: {loss:e}, Test Acc: {test_acc:e}, Time: {end-start:.4f}')
        # Show the plot
        plt.show()
    
    train_loader, test_loader = prepare_training_data_and_models(sample_data_size=10000, just_data=True)

torch.save(model, 'gcm.torch')
del train_loader
del test_loader

# Evaluate

Now, we write some basic code to take a Networkx graph and predict the classes.

In [ ]:
def predict_mult(ids, sequences, verbose=False):    
    protein_dataset = ProteinDataset(sequences, torch.zeros((len(ids),1500,1)), training=False)
    loader = DataLoader(protein_dataset, batch_size=1024, shuffle=False)

    if verbose:
        print("Evaluating sequences")
        print("Total sequences: " + str(len(ids)))
        
    # Set the model to evaluation mode
    model.eval()
    predictions = []  # initialize an empty list to collect all predictions
    
    # Forward pass through the model
    with torch.no_grad():
        
        for i,batch in enumerate(loader):
            batch = batch.to(device)  # move batch to the device (GPU or CPU)
            outputs = model(batch)  # forward pass
            # process the outputs (e.g., apply a threshold in case of binary classification, etc.)
            batch_predictions = (outputs > 0.5).float()
            if verbose:
                print("Batch output " + str(i) + ": ")
                print(batch_predictions)
            # Detach the predictions from the computation graph and convert to a numpy array.
            # Then add them to the list of all predictions.
            predictions.extend(batch_predictions.cpu().numpy().tolist())

        if verbose:
            print("Model prediction: ")
            print(predictions)
        #output = model(x=data.x, edge_index=data.edge_index, edge_weight=edge_weight)

    return ids, predictions

def predict(sequence, verbose=False):
    g = protein_recurrence_network(sequence)
    # Convert the NetworkX graph to a PyTorch Geometric Data object
    data = from_networkx_to_data(g, torch.zeros((1500,1)))
    
    if verbose:
        print("Data: ")
        print(data)

    # Move data to the appropriate device
    data = data.to(device)

    # Set the model to evaluation mode
    model.eval()

    # Forward pass through the model
    with torch.no_grad():
        output = model(data)
        if verbose:
            print("Model prediction: ")
            print(output)
        #output = model(x=data.x, edge_index=data.edge_index, edge_weight=edge_weight)

    # The output is log-probabilities, use softmax to get probabilities
    out_probs = torch.sigmoid(output)
    
    # In a multi-label classification, an element is considered to be predicted as '1'
    # if its probability is greater than or equal to 0.5.
    out_preds = (out_probs > 0.5).float()

    return out_preds

In [ ]:
test_seq = 'YAWVHDHCNWCLERVGTVHDEKWTMEDPMVPKHCECNEPIQAWTQDNDLYNFCITLQICANNNRDGGNLIGRVDQLALQKRLLDQNWHKNCTPPSVVTRCNCATCEEKVETRVHFMKIMGESGWDGWVMYLPLGQQICYSIPHRQHKCWWSIWFVDFRLLPIGERNDTLSCFMIIKLEWIFDVCHLHDKSIEIAEAGVQIQVWAQQICSGTYIEGWLDWENSPIACPDGDYWSNWTAMVVAFCECRKRMCSVIPQTKFYAFHMVHNKWWLQWFYTSHEGKKIVGIEYHQGSCYYKWTKGEKHMHMNVEQRQWGADQVVHTPFNAWACWMHTAKHGDCNVPGQHWGMGWFRDDDELAAYGHHTEHGDATQLLFGTCYVCPLNIVYLMGTLVLPRHQHVRNKRPMVDRDMVYTRDKIIQELVWHGSYYILRPLMMRNTQHIKYLVYRGFHSIPKKDIAQPKRRGNKVGDKHVWKQFWIGSQPNHSKNTLMDLWHSAFMKWCNDFPDEPTKYVAGWPNIHPMGLALCLHPPSPRREKWDKHMFTPDYDHMSSWPEFAAHMDDCWVHNTFQIPWFWPNHHGRFHAQNVFLVWFTTGIKAGLDLMNICMIWGDKPRTHKIIFHHMRDWKNHPIEFTMKMEEKYHWDGKFDYHLPFRYWDRMSIRHDNYKTSWPCGWPSWHYALIELTCYGEMEIYGWADQEYRQCDQTFHMQLQMQNMTKLIRWYTHLGCDSVMCVQSALHTLKEWKRPTRMIKNGLGLILNLMVMACAGHFTISDGFLLLESPWKWGNSYSGNSVEGILAVKGVDYDFDGDYAWEQRSNQQWVGGLMQCILAGLPRLNLEMFRQESMWANTYGMPSYKQTDFFHSNLIRTKRMDEMSHAMVRGWTGILVNFTHHNWTWYMSLLCQSAYGTARTIFTGGNPAADDNRDLDHEDDEVYDVWIDSECTAQWLSMPVGIPNYGFCTCQAKQWINRGIPAGKLFMEVCPWMNDNQKGASSHCYIIEWCS'
t = predict(test_seq)
display(t)
test_seqs = ['YAWVHDHCNWCLERVGTVHDEKWTMEDPMVPKHCECNEPIQAWTQDNDLYNFCITLQICANNNRDGGNLIGRVDQLALQKRLLDQNWHKNCTPPSVVTRCNCATCEEKVETRVHFMKIMGESGWDGWVMYLPLGQQICYSIPHRQHKCWWSIWFVDFRLLPIGERNDTLSCFMIIKLEWIFDVCHLHDKSIEIAEAGVQIQVWAQQICSGTYIEGWLDWENSPIACPDGDYWSNWTAMVVAFCECRKRMCSVIPQTKFYAFHMVHNKWWLQWFYTSHEGKKIVGIEYHQGSCYYKWTKGEKHMHMNVEQRQWGADQVVHTPFNAWACWMHTAKHGDCNVPGQHWGMGWFRDDDELAAYGHHTEHGDATQLLFGTCYVCPLNIVYLMGTLVLPRHQHVRNKRPMVDRDMVYTRDKIIQELVWHGSYYILRPLMMRNTQHIKYLVYRGFHSIPKKDIAQPKRRGNKVGDKHVWKQFWIGSQPNHSKNTLMDLWHSAFMKWCNDFPDEPTKYVAGWPNIHPMGLALCLHPPSPRREKWDKHMFTPDYDHMSSWPEFAAHMDDCWVHNTFQIPWFWPNHHGRFHAQNVFLVWFTTGIKAGLDLMNICMIWGDKPRTHKIIFHHMRDWKNHPIEFTMKMEEKYHWDGKFDYHLPFRYWDRMSIRHDNYKTSWPCGWPSWHYALIELTCYGEMEIYGWADQEYRQCDQTFHMQLQMQNMTKLIRWYTHLGCDSVMCVQSALHTLKEWKRPTRMIKNGLGLILNLMVMACAGHFTISDGFLLLESPWKWGNSYSGNSVEGILAVKGVDYDFDGDYAWEQRSNQQWVGGLMQCILAGLPRLNLEMFRQESMWANTYGMPSYKQTDFFHSNLIRTKRMDEMSHAMVRGWTGILVNFTHHNWTWYMSLLCQSAYGTARTIFTGGNPAADDNRDLDHEDDEVYDVWIDSECTAQWLSMPVGIPNYGFCTCQAKQWINRGIPAGKLFMEVCPWMNDNQKGASSHCYIIEWCS','YAWVHDHCNVPKHCECNEPIQAWTQDNDLYNFCITLQICANNNRDGGNLIGRVDQLALQKRLLDQNWHKNCTPPSVVTRCNCATCEEKVETRVHFMKIMGESGWDGWVMYLPLGQQICYSIPHRQHKCWWSIWFVDFRLLPIGERNDTLSCFMIIKLEWIFDVCHLHDKSIEIAEAGVQIQVWAQQICSGTYIEGWLDWENSPIACPDGDYWSNWTAMVVAFCECRKRMCSVIPQTKFYAFHMVHNKWWLQWFYTSHEGKKIVGIEYHQGSCYYKWTKGEKHMHMNVEQRQWGADQVVHTPFNAWACWMHTAKHGDCNVPGQHWGMGWFRDDDELAAYGHHTEHGDATQLLFGTCYVCPLNIVYLMGTLVLPRHQHVRNKRPMVDRDMVYTRDKIIQELVWHGSYYILRPLMMRNTQHIKYLVYRGFHSIPKKDIAQPKRRGNKVGDKHVWKQFWIGSQPNHSKNTLMDLWHSAFMKWCNDFPDEPTKYVAGWPNIHPMGLALCLHPPSPRREKWDKHMFTPDYDHMSSWPEFAAHMDDCWVHNTFQIPWFWPNHHGRFHAQNVFLVWFTTGIKAGLDLMNICMIWGDKPRTHKIIFHHMRDWKNHPIEFTMKMEEKYHWDGKFDYHLPFRYWDRMSIRHDNYKTSWPCGWPSWHYALIELTCYGEMEIYGWADQEYRQCDQTFHMQLQMQNMTKLIRWYTHLGCDSVMCVQSALHTLKEWKRPTRMIKNGLGLILNLMVMACAGHFTISDGFLLLESPWKWGNSYSGNSVEGILAVKGVDYDFDGDYAWEQRSNQQWVGGLMQCILAGLPRLNLEMFRQESMWANTYGMPSYKQTDFFHSNLIRTKRMDEMSHAMVRGWTGILVNFTHHNWTWYMSLLCQSAYGTARTIFTGGNPAADDNRDLDHEDDEVYDVWIDSECTAQWLSMPVGIPNYGFCTCQAKQWINRGIPAGKLFMEVCPWMNDNQKGASSHCYIIEWCS']
ids,ts = predict_mult(['t1','t2'],test_seqs)
display(ts)
#torch.nonzero(predict(test_seq))
#labels_df.iloc[np.where(train_protein_ids=='P20536')[0][0]]

In [ ]:
np.array(ts)

In [ ]:
t

# Submission

For submission we will use the protein embeddings of the test data created by [Sergei Fironov](https://www.kaggle.com/sergeifironov) using the Rost Lab's T5 protein language model.

In [ ]:
# Parse the fasta file
kaggle_input_embeds = '/kaggle/input/t5embeds'
test_protein_ids = set(np.load(kaggle_input_embeds + '/test_ids.npy'))
test_records = SeqIO.parse(kaggle_input_data + '/Test (Targets)/testsuperset.fasta', format='fasta')

pred_ids, predictions = predict_mult(*list(zip(*[(record.id, str(record.seq)) for record in test_records if record.id in test_protein_ids[:100]])), verbose=True)

In [ ]:
predictions

From the predictions we will create the submission data frame.

**Note**: This will take atleast **15 to 20** minutes to finish.

In [ ]:
# Reference: https://www.kaggle.com/code/alexandervc/baseline-multilabel-to-multitarget-binary

df_submission = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])
l = []
for k in list(test_protein_ids):
    l += [ k] * predictions.shape[1]

df_submission['Protein Id'] = l
df_submission['GO Term Id'] = labels * predictions.shape[0]
df_submission['Prediction'] = predictions.ravel()
df_submission.to_csv("submission.tsv",header=False, index=False, sep="\t")

In [ ]:
df_submission